In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('datasets/train.csv')
print(df.head())

   stock_id  date_id  seconds_in_bucket  imbalance_size   
0         0        0                  0      3180602.69  \
1         1        0                  0       166603.91   
2         2        0                  0       302879.87   
3         3        0                  0     11917682.27   
4         4        0                  0       447549.96   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price   
0                        1         0.999812   13380276.64        NaN  \
1                       -1         0.999896    1642214.25        NaN   
2                       -1         0.999561    1819368.03        NaN   
3                       -1         1.000171   18389745.62        NaN   
4                       -1         0.999532   17860614.95        NaN   

   near_price  bid_price  bid_size  ask_price   ask_size  wap    target   
0         NaN   0.999812  60651.50   1.000026    8493.03  1.0 -3.029704  \
1         NaN   0.999896   3233.04   1.000660   20605.09  1.0 

In [10]:
df = pd.read_csv('datasets/train.csv')
df = df[df['stock_id'] < 20]

In [11]:
df['momentum'] = df.groupby(by=['stock_id', 'date_id'])['wap'].transform(lambda x: x - x.shift(3))

In [12]:
df['volume_rolling_mean'] = df.groupby(by=['stock_id', 'date_id'])['matched_size'].transform(lambda x: x.rolling(5).mean())

In [13]:
df['volume_spike'] = df['matched_size'] > 2*df['volume_rolling_mean']

In [15]:
df['maximal_wap_today'] = df.groupby(by=['stock_id', 'date_id'])['wap'].transform(lambda x: x.max())
df['minimal_wap_today'] = df.groupby(by=['stock_id', 'date_id'])['wap'].transform(lambda x: x.min())

thresh = 0.01
df['position_in_daily_range'] = (df['wap']- df['minimal_wap_today'])/(df['maximal_wap_today']-df['minimal_wap_today'] + thresh)


In [18]:
df['first_time_today_stock'] = df.groupby(by=['stock_id', 'date_id'])['seconds_in_bucket'].transform('min')
df['seconds_since_open'] = df['seconds_in_bucket'] - df['first_time_today_stock']
